In [5]:
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
from bokeh.models import Arrow, NormalHead
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
%run Initialiser.ipynb

def SFEE(Qdot,Wdot,mdot,h2,c2,z2,h1,c1,z1):
    g=9.81
    if Qdot==None:
        Qdotsolved=Wdot+mdot*((h2+(0.5*(c2**2))+(g*z2))-(h1+(0.5*(c1**2))+(g*z1)))
        return Qdotsolved
    elif Wdot==None:
        Wdotsolved=Qdot-(mdot*((h2+(0.5*(c2**2))+(g*z2))-(h1+(0.5*(c1**2))+(g*z1))))
        return Qdotsolved
    elif mdot==None:
        mdotsolved=(Qdot-Wdot)/((h2+(0.5*(c2**2))+(g*z2))-(h1+(0.5*(c1**2))+(g*z1)))
        return mdotsolved
    elif h1==None:
        h1solved=h2+(0.5*(c2**2))+9.81*z2-(0.5*(c1**2))-9.81*z1+(Wdot-Qdot)/mdot
        return h1solved
    elif c1==None:
        c1solved=sqrt(2*(h2+0.5*(c2**2)+9.81*z2-h1-9.81*z1+(Wdot-Qdot)/mdot))
        return c1solved
    elif z1==None:
        z1solved=(1/9.81)*(h2+0.5*(c2**2)+9.81*z2-h1-0.5*(c1**2)+(Wdot-Qdot)/mdot)
        return z1solved
    elif h2==None:
        h2solved=((Qdot-Wdot)/mdot)+h1+((c1**2)/2)+9.81*z1-((c2**2)/2)-9.81*z2
        return h2solved
    elif c2==None:
        c2solved=sqrt(2*(((Qdot-Wdot)/mdot)+h1+0.5*(c1**2)+9.81*z1-h2-9.81*z2))
        return c2solved
    elif z2==None:
        z2solved=(1/9.81)*(((Qdot-Wdot)/mdot)+h1+0.5*(c1**2)+9.81*z1-h2-0.5*(c2**2))
        return z2solved

#only for steam:
def checksteamstate(propslist,statenum): #checks if steam at a given state is subcooled, superheated or wet.
    steamstate=""
    if propslist[0][statenum-1]!="": #first trying to use quality/x. Note that "" is used instead of zero for quality, as quality CAN be zero. 
        if propslist[0][statenum-1]!=-1:
            if propslist[0][statenum-1]==1:
                steamstate="saturated vapour"
            elif propslist[0][statenum-1]==0:
                steamstate="saturated liquid"
            elif float(propslist[0][statenum-1])<1 and float(propslist[0][statenum-1])>0:
                steamstate="wet steam"
        elif propslist[1][statenum-1]!=0 and propslist[2][statenum-1]!=0:
            if float(propslist[2][statenum-1])>CP.PropsSI('T','Q',0.5,'P',propslist[1][statenum-1],"water"):
                steamstate="superheated"
            elif float(propslist[2][statenum-1])<CP.PropsSI('T','Q',0.5,'P',propslist[1][statenum-1],"water"):
                steamstate="subcooled"
    elif propslist[1][statenum-1]!=0 and propslist[2][statenum-1]!=0:
            if float(propslist[2][statenum-1])>CP.PropsSI('T','Q',0.5,'P',propslist[1][statenum-1],"water"):
                steamstate="superheated"
            elif float(propslist[2][statenum-1])<CP.PropsSI('T','Q',0.5,'P',propslist[1][statenum-1],"water"):
                steamstate="subcooled"
    elif propslist[1][statenum-1]!=0:
        if propslist[3][statenum-1]!=0:
            u_g= CP.PropsSI("U","P",propslist[1][statenum-1],"Q",1.0,"water")
            u_f= CP.PropsSI("U","P",propslist[1][statenum-1],"Q",0,'water')
            if float(propslist[3][statenum-1])>u_g:
                steamstate="superheated"
            elif float(propslist[3][statenum-1])<u_f:
                steamstate="subcooled"
            elif propslist[3][statenum-1]==u_g:
                steamstate="saturated vapour"
            elif propslist[3][statenum-1]==u_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
        elif propslist[4][statenum-1]!=0:
            s_g= CP.PropsSI("S","P",propslist[1][statenum-1],"Q",1.0,'water')
            s_f= CP.PropsSI("S","P",propslist[1][statenum-1],"Q",0,'water')
            if float(propslist[4][statenum-1])>s_g:
                steamstate="superheated"
            elif float(propslist[4][statenum-1])<s_f:
                steamstate="subcooled"
            elif propslist[4][statenum-1]==s_g:
                steamstate="saturated vapour"
            elif propslist[4][statenum-1]==s_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
        elif propslist[5][statenum-1]!=0:
            h_g= CP.PropsSI("H","P",propslist[1][statenum-1],"Q",1.0,'water')
            h_f= CP.PropsSI("H","P",propslist[1][statenum-1],"Q",0,'water')
            if float(propslist[5][statenum-1])>h_g:
                steamstate="superheated"
            elif float(propslist[5][statenum-1])<h_f:
                steamstate="subcooled"
            elif propslist[5][statenum-1]==h_g:
                steamstate="saturated vapour"
            elif propslist[5][statenum-1]==h_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
        elif propslist[6][statenum-1]!=0:
            v_g= 1/(CP.PropsSI("D","P",propslist[1][statenum-1],"Q",1.0,'water'))
            v_f= 1/(CP.PropsSI("D","P",propslist[1][statenum-1],"Q",0,'water'))
            if float(propslist[6][statenum-1])>v_g:
                steamstate="superheated"
            elif float(propslist[6][statenum-1])<v_f:
                steamstate="subcooled"
            elif propslist[6][statenum-1]==v_g:
                steamstate="saturated vapour"
            elif propslist[6][statenum-1]==v_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
    elif propslist[2][statenum-1]!=0:
        if propslist[3][statenum-1]!=0:
            u_g= CP.PropsSI("U","T",propslist[2][statenum-1],"Q",1.0,'water')
            u_f= CP.PropsSI("U","T",propslist[2][statenum-1],"Q",0,'water')
            if float(propslist[3][statenum-1])>u_g:
                steamstate="superheated"
            elif float(propslist[3][statenum-1])<u_f:
                steamstate="subcooled"
            elif propslist[3][statenum-1]==u_g:
                steamstate="saturated vapour"
            elif propslist[3][statenum-1]==u_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
        elif propslist[4][statenum-1]!=0:
            s_g= CP.PropsSI("S","T",propslist[2][statenum-1],"Q",1.0,'water')
            s_f= CP.PropsSI("S","T",propslist[2][statenum-1],"Q",0,'water')
            if float(propslist[4][statenum-1])>s_g:
                steamstate="superheated"
            elif float(propslist[4][statenum-1])<s_f:
                steamstate="subcooled"
            elif propslist[4][statenum-1]==s_g:
                steamstate="saturated vapour"
            elif propslist[4][statenum-1]==s_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
        elif propslist[5][statenum-1]!=0:
            h_g= CP.PropsSI("H","T",propslist[2][statenum-1],"Q",1.0,'water')
            h_f= CP.PropsSI("H","T",propslist[2][statenum-1],"Q",0,'water')
            if float(propslist[5][statenum-1])>h_g:
                steamstate="superheated"
            elif float(propslist[5][statenum-1])<h_f:
                steamstate="subcooled"
            elif propslist[5][statenum-1]==h_g:
                steamstate="saturated vapour"
            elif propslist[5][statenum-1]==h_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
        elif propslist[6][statenum-1]!=0:
            v_g= 1/(CP.PropsSI("D","T",propslist[2][statenum-1],"Q",1.0,'water'))
            v_f= 1/(CP.PropsSI("D","T",propslist[2][statenum-1],"Q",0,'water'))
            if float(propslist[6][statenum-1])>v_g:
                steamstate="superheated"
            elif float(propslist[6][statenum-1])<v_f:
                steamstate="subcooled"
            elif propslist[6][statenum-1]==v_g:
                steamstate="saturated vapour"
            elif propslist[6][statenum-1]==v_f:
                steamstate="saturated liquid"
            else:
                steamstate="wet steam"
    return steamstate


def checkfullydefined(propslist,statenum): #note: does not recognise overdefinition
    sum=0
    fullydefined=False
    if checksteamstate(propslist,statenum)=="superheated" or checksteamstate(propslist,statenum)=="subcooled" : #any two properties can fully define the state. 
        sum=0
        sum2=0
        if propslist[0][statenum-1]!="": #dryness fraction is defined
            for i in range(1,7):
                if propslist[i][statenum-1]!=0:
                    sum2+=1
                sum=sum2+1

        elif propslist[0][statenum-1]=="": #dryness fraction not defined
            for i in range(1,7):
                if propslist[i][statenum-1]!=0:
                    sum+=1
        if sum>=2:
            fullydefined=True 

    elif checksteamstate(propslist,statenum)=="saturated vapour" or checksteamstate(propslist,statenum)=="saturated liquid" or checksteamstate(propslist,statenum)=="wet steam":
        sum=0
        if propslist[1][statenum-1]!=0 and propslist[2][statenum-1]!=0 and propslist[0][statenum-1]=="" and propslist[3][statenum-1]==0 and propslist[4][statenum-1]==0 and propslist[5][statenum-1]==0 and propslist[6][statenum-1]==0: #only P and T given
            fullydefined=False #underdefined
        else:
            for i in range(0,6):
                if propslist[i][statenum-1]!=0:
                    sum+=1
            if sum>=2:
                fullydefined=True
    return fullydefined


def checkfullydefidealgas(propslist,statenum): #state is fixed by any two props
    sum=0
    fullydefined=False
    for i in range(0,6):
        if propslist[i][statenum-1]!=0:
            sum+=1
    if sum>=2: 
        fullydefined=True

def getallproperties(propslist, statenum): #note that the propslist is arranged as follows: [x...,P...,T...,u...,s...,h...,v...]. This function tries to obtain all of the properties, given a properly defined state.
#COMBOS NOT SUPPORTED: T and u, T and h, h and s, h and x, s and x, u and h, u and s
#note that if there is insufficient info then the function will return the list as it was.
    if propslist[1][statenum-1]!=0 and propslist[6][statenum-1]!=0: #P and v given only.FULLY DEFINED IRRESPECTIVE OF WHERE WE ARE. 
        propslist[0][statenum-1]=CP.PropsSI("Q","P",propslist[1][statenum-1],"D",1/(propslist[6][statenum-1]),"water")
        propslist[2][statenum-1]=CP.PropsSI("T","P",propslist[1][statenum-1],"D",1/(propslist[6][statenum-1]),"water")
        propslist[3][statenum-1]=CP.PropsSI("U","P",propslist[1][statenum-1],"D",1/(propslist[6][statenum-1]),"water")
        propslist[4][statenum-1]=CP.PropsSI("S","P",propslist[1][statenum-1],"D",1/(propslist[6][statenum-1]),"water")
        propslist[5][statenum-1]=CP.PropsSI("H","P",propslist[1][statenum-1],"D",1/(propslist[6][statenum-1]),"water")
    elif propslist[1][statenum-1]!=0 and propslist[5][statenum-1]!=0: #P and h given only. FULLY DEFINED IRRESPECTIVE OF WHERE WE ARE
        propslist[0][statenum-1]=CP.PropsSI("Q","P",propslist[1][statenum-1],"H",propslist[5][statenum-1],"water")
        propslist[2][statenum-1]=CP.PropsSI("T","P",propslist[1][statenum-1],"H",propslist[5][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","P",propslist[1][statenum-1],"H",propslist[5][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","P",propslist[1][statenum-1],"H",propslist[5][statenum-1],"water")
        propslist[6][statenum-1]=1/(CP.PropsSI("D","P",propslist[1][statenum-1],"H",propslist[5][statenum-1],"water"))
    elif propslist[1][statenum-1]!=0 and propslist[0][statenum-1]!="" and propslist[0][statenum-1]!=-1: #i.e within the dome (P and x). P and x only define the state if within the vapour dome. 
        propslist[2][statenum-1]=CP.PropsSI("T","P",propslist[1][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","P",propslist[1][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","P",propslist[1][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","P",propslist[1][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[6][statenum-1]=1/(CP.PropsSI("D","P",propslist[1][statenum-1],"Q",propslist[0][statenum-1],"water"))
    elif propslist[1][statenum-1]!=0 and propslist[4][statenum-1]!=0: #P and s
        propslist[0][statenum-1]=CP.PropsSI("Q","P",propslist[1][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[2][statenum-1]=CP.PropsSI("T","P",propslist[1][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","P",propslist[1][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","P",propslist[1][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[6][statenum-1]=1/(CP.PropsSI("D","P",propslist[1][statenum-1],"S",propslist[4][statenum-1],"water"))
    elif propslist[1][statenum-1]!=0 and propslist[3][statenum-1]!=0: #P and u
        propslist[0][statemum-1]=CP.PropsSI("Q","P",propslist[1][statenum-1],"U",propslist[3][statenum-1],"water")
        propslist[2][statemum-1]=CP.PropsSI("T","P",propslist[1][statenum-1],"U",propslist[3][statenum-1],"water")
        propslist[4][statemum-1]=CP.PropsSI("S","P",propslist[1][statenum-1],"U",propslist[3][statenum-1],"water")
        propslist[5][statemum-1]=CP.PropsSI("H","P",propslist[1][statenum-1],"U",propslist[3][statenum-1],"water")
        propslist[6][statemum-1]=1/(CP.PropsSI("D","P",propslist[1][statenum-1],"U",propslist[3][statenum-1],"water"))
    elif propslist[1][statenum-1]!=0 and propslist[2][statenum-1]!=0 and (checksteamstate(propslist,statenum)=="superheated" or checksteamstate(propslist,statenum)=="subcooled") : #P and T and superheated state.
        propslist[0][statenum-1]=CP.PropsSI("Q","P",propslist[1][statenum-1],"T",propslist[2][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","P",propslist[1][statenum-1],"T",propslist[2][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","P",propslist[1][statenum-1],"T",propslist[2][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","P",propslist[1][statenum-1],"T",propslist[2][statenum-1],"water")
        propslist[6][statenum-1]=1/(CP.PropsSI("D","P",propslist[1][statenum-1],"T",propslist[2][statenum-1],"water"))
    elif propslist[1][statenum-1]!=0 and propslist[2][statenum-1]!=0 and checksteamstate(propslist,statenum)=="wet steam":
        print("state is under defined")
    elif propslist[2][statenum-1]!=0 and propslist[0][statenum-1]!="" and propslist[0][statenum-1]!=-1: #inside vapour dome, T and x
        propslist[1][statenum-1]=CP.PropsSI("P","T",propslist[2][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","T",propslist[2][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","T",propslist[2][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","T",propslist[2][statenum-1],"Q",propslist[0][statenum-1],"water")
        propslist[6][statenum-1]=1/(CP.PropsSI("D","T",propslist[2][statenum-1],"Q",propslist[0][statenum-1],"water"))
    elif propslist[2][statenum-1]!=0 and propslist[4][statenum-1]!=0: #T and s given
        propslist[0][statenum-1]=CP.PropsSI("Q","T",propslist[2][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[1][statenum-1]=CP.PropsSI("P","T",propslist[2][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","T",propslist[2][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","T",propslist[2][statenum-1],"S",propslist[4][statenum-1],"water")
        propslist[6][statenum-1]=1/(CP.PropsSI("D","T",propslist[2][statenum-1],"S",propslist[4][statenum-1],"water"))
    elif propslist[2][statenum-1]!=0 and propslist[6][statenum-1]!=0: #T and v given
        propslist[0][statenum-1]=CP.PropsSI("Q","T",propslist[2][statenum-1],"D",1/( propslist[6][statenum-1]),"water")
        propslist[1][statenum-1]=CP.PropsSI("P","T",propslist[2][statenum-1],"D",1/( propslist[6][statenum-1]),"water")
        propslist[3][statenum-1]=CP.PropsSI("U","T",propslist[2][statenum-1],"D",1/( propslist[6][statenum-1]),"water")
        propslist[4][statenum-1]=CP.PropsSI("S","T",propslist[2][statenum-1],"D",1/( propslist[6][statenum-1]),"water")
        propslist[5][statenum-1]=CP.PropsSI("H","T",propslist[2][statenum-1],"D",1/( propslist[6][statenum-1]),"water")
    elif propslist[6][statenum-1]!=0 and propslist[5][statenum-1]!=0: #v and h
        propslist[0][statenum-1]=CP.PropsSI("Q","D",1/( propslist[6][statenum-1]),"H",propslist[5][statenum-1],"water")
        propslist[1][statenum-1]=CP.PropsSI("P","D",1/( propslist[6][statenum-1]),"H",propslist[5][statenum-1],"water")
        propslist[2][statenum-1]=CP.PropsSI("T","D",1/( propslist[6][statenum-1]),"H",propslist[5][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","D",1/( propslist[6][statenum-1]),"H",propslist[5][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","D",1/( propslist[6][statenum-1]),"H",propslist[5][statenum-1],"water")
    elif propslist[6][statenum-1]!=0 and propslist[4][statenum-1]!=0: #v and s
        propslist[0][statenum-1]=CP.PropsSI("Q","D",1/(propslist[6][statenum-1]),"S",propslist[4][statenum-1],"water")
        propslist[1][statenum-1]=CP.PropsSI("P","D",1/(propslist[6][statenum-1]),"S",propslist[4][statenum-1],"water")
        propslist[2][statenum-1]=CP.PropsSI("T","D",1/(propslist[6][statenum-1]),"S",propslist[4][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","D",1/(propslist[6][statenum-1]),"S",propslist[4][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","D",1/(propslist[6][statenum-1]),"S",propslist[4][statenum-1],"water")
    elif propslist[6][statenum-1]!=0 and propslist[3][statenum-1]!=0: #v and u
        propslist[0][statenum-1]=CP.PropsSI("Q","D",1/(propslist[6][statenum-1]),"U",propslist[3][statenum-1],"water")
        propslist[1][statenum-1]=CP.PropsSI("P","D",1/(propslist[6][statenum-1]),"U",propslist[3][statenum-1],"water")
        propslist[2][statenum-1]=CP.PropsSI("T","D",1/(propslist[6][statenum-1]),"U",propslist[3][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","D",1/(propslist[6][statenum-1]),"U",propslist[3][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","D",1/(propslist[6][statenum-1]),"U",propslist[3][statenum-1],"water")
    elif propslist[6][statenum-1]!=0 and propslist[0][statenum-1]!="" and propslist[0][statenum-1]!=-1: #v and x, within vapour dome.
        propslist[1][statenum-1]=CP.PropsSI("P","D",1/(propslist[6][statenum-1]),"Q",propslist[0][statenum-1],"water")
        propslist[2][statenum-1]=CP.PropsSI("T","D",1/(propslist[6][statenum-1]),"Q",propslist[0][statenum-1],"water")
        propslist[3][statenum-1]=CP.PropsSI("U","D",1/(propslist[6][statenum-1]),"Q",propslist[0][statenum-1],"water")
        propslist[4][statenum-1]=CP.PropsSI("S","D",1/(propslist[6][statenum-1]),"Q",propslist[0][statenum-1],"water")
        propslist[5][statenum-1]=CP.PropsSI("H","D",1/(propslist[6][statenum-1]),"Q",propslist[0][statenum-1],"water")
    return propslist

def getallgasproperties(propslist,statenum): #propslist for ideal gas: [P,T,u,s,h,v]
    if propslist[0][statenum-1]!=0 and propslist[5][statenum-1]!=0: #P and v given only.FULLY DEFINED IRRESPECTIVE OF WHERE WE ARE. 
        propslist[1][statenum-1]=CP.PropsSI("T","P",propslist[0][statenum-1],"D",1/(propslist[5][statenum-1]),fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","P",propslist[0][statenum-1],"D",1/(propslist[5][statenum-1]),fluid)
        propslist[3][statenum-1]=CP.PropsSI("S","P",propslist[0][statenum-1],"D",1/(propslist[5][statenum-1]),fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","P",propslist[0][statenum-1],"D",1/(propslist[5][statenum-1]),fluid)

    elif propslist[0][statenum-1]!=0 and propslist[4][statenum-1]!=0: #P and h given only. FULLY DEFINED IRRESPECTIVE OF WHERE WE ARE
        propslist[1][statenum-1]=CP.PropsSI("T","P",propslist[0][statenum-1],"H",propslist[4][statenum-1],fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","P",propslist[0][statenum-1],"H",propslist[4][statenum-1],fluid)
        propslist[3][statenum-1]=CP.PropsSI("S","P",propslist[0][statenum-1],"H",propslist[4][statenum-1],fluid)
        propslist[5][statenum-1]=1/(CP.PropsSI("D","P",propslist[0][statenum-1],"H",propslist[4][statenum-1],fluid))

    elif propslist[0][statenum-1]!=0 and propslist[3][statenum-1]!=0: #P and s
        propslist[1][statenum-1]=CP.PropsSI("T","P",propslist[0][statenum-1],"S",propslist[3][statenum-1],fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","P",propslist[0][statenum-1],"S",propslist[3][statenum-1],fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","P",propslist[0][statenum-1],"S",propslist[3][statenum-1],fluid)
        propslist[5][statenum-1]=1/(CP.PropsSI("D","P",propslist[0][statenum-1],"S",propslist[3][statenum-1],fluid))

    elif propslist[0][statenum-1]!=0 and propslist[2][statenum-1]!=0: #P and u
        propslist[1][statemum-1]=CP.PropsSI("T","P",propslist[0][statenum-1],"U",propslist[2][statenum-1],fluid)
        propslist[3][statemum-1]=CP.PropsSI("S","P",propslist[0][statenum-1],"U",propslist[2][statenum-1],fluid)
        propslist[4][statemum-1]=CP.PropsSI("H","P",propslist[0][statenum-1],"U",propslist[2][statenum-1],fluid)
        propslist[5][statemum-1]=1/(CP.PropsSI("D","P",propslist[0][statenum-1],"U",propslist[2][statenum-1],fluid))

    elif propslist[0][statenum-1]!=0 and propslist[1][statenum-1]!=0: #P and T
        propslist[2][statenum-1]=CP.PropsSI("U","P",propslist[0][statenum-1],"T",propslist[1][statenum-1],fluid)
        propslist[3][statenum-1]=CP.PropsSI("S","P",propslist[0][statenum-1],"T",propslist[1][statenum-1],fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","P",propslist[0][statenum-1],"T",propslist[1][statenum-1],fluid)
        propslist[5][statenum-1]=1/(CP.PropsSI("D","P",propslist[0][statenum-1],"T",propslist[1][statenum-1],fluid))

    elif propslist[1][statenum-1]!=0 and propslist[3][statenum-1]!=0: #T and s given
        propslist[0][statenum-1]=CP.PropsSI("P","T",propslist[1][statenum-1],"S",propslist[3][statenum-1],fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","T",propslist[1][statenum-1],"S",propslist[3][statenum-1],fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","T",propslist[1][statenum-1],"S",propslist[3][statenum-1],fluid)
        propslist[5][statenum-1]=1/(CP.PropsSI("D","T",propslist[1][statenum-1],"S",propslist[3][statenum-1],fluid))

    elif propslist[1][statenum-1]!=0 and propslist[5][statenum-1]!=0: #T and v given
        propslist[0][statenum-1]=CP.PropsSI("P","T",propslist[1][statenum-1],"D",1/( propslist[5][statenum-1]),fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","T",propslist[1][statenum-1],"D",1/( propslist[5][statenum-1]),fluid)
        propslist[3][statenum-1]=CP.PropsSI("S","T",propslist[1][statenum-1],"D",1/( propslist[5][statenum-1]),fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","T",propslist[1][statenum-1],"D",1/( propslist[5][statenum-1]),fluid)
    
    elif propslist[5][statenum-1]!=0 and propslist[4][statenum-1]!=0: #v and h
        propslist[0][statenum-1]=CP.PropsSI("P","D",1/( propslist[5][statenum-1]),"H",propslist[4][statenum-1],fluid)
        propslist[1][statenum-1]=CP.PropsSI("T","D",1/( propslist[5][statenum-1]),"H",propslist[4][statenum-1],fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","D",1/( propslist[5][statenum-1]),"H",propslist[4][statenum-1],fluid)
        propslist[3][statenum-1]=CP.PropsSI("S","D",1/( propslist[5][statenum-1]),"H",propslist[4][statenum-1],fluid)
    
    elif propslist[5][statenum-1]!=0 and propslist[3][statenum-1]!=0: #v and s
        propslist[0][statenum-1]=CP.PropsSI("P","D",1/(propslist[5][statenum-1]),"S",propslist[3][statenum-1],fluid)
        propslist[1][statenum-1]=CP.PropsSI("T","D",1/(propslist[5][statenum-1]),"S",propslist[3][statenum-1],fluid)
        propslist[2][statenum-1]=CP.PropsSI("U","D",1/(propslist[5][statenum-1]),"S",propslist[3][statenum-1],fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","D",1/(propslist[5][statenum-1]),"S",propslist[3][statenum-1],fluid)
    
    elif propslist[5][statenum-1]!=0 and propslist[2][statenum-1]!=0: #v and u
        propslist[0][statenum-1]=CP.PropsSI("P","D",1/(propslist[5][statenum-1]),"U",propslist[2][statenum-1],fluid)
        propslist[1][statenum-1]=CP.PropsSI("T","D",1/(propslist[5][statenum-1]),"U",propslist[2][statenum-1],fluid)
        propslist[3][statenum-1]=CP.PropsSI("S","D",1/(propslist[5][statenum-1]),"U",propslist[2][statenum-1],fluid)
        propslist[4][statenum-1]=CP.PropsSI("H","D",1/(propslist[5][statenum-1]),"U",propslist[2][statenum-1],fluid)
    
    return propslist



def checkallstatesdefined(propslist):
    allstatesdefined=False
    counter=0
    for i in range(numberofstates):
        if checkfullydefined(propslist,i+1)==True:
            counter+=1
    if counter==numberofstates:
        allstatesdefined=True
    elif counter!=numberofstates:
        allstatesdefined=False
    return allstatesdefined


def checkheatandworkdefined(qtransfer,wtransfer):  
    heatandworkdefined=False
    qcounter=0
    wcounter0
    for i in range(numberofstates):
        if qtransfer[i]!="":
            qcounter+=1
        if wtransfer[i]!="":
            wcounter+=1
    if qcounter==numberofstates and wcounter==numberofstates:
        heatandworkdefined=True
    else:
        heatandworkdefined=False


